In [186]:
import os

from datetime import date, datetime, timedelta
from tqdm.auto import tqdm
from math import sqrt

import numpy as np
import pandas as pd

import networkx as nx

In [298]:
class Roads:
    def __init__(self, data_folder):
        self.data_folder = data_folder
        if data_folder[-1] != '/':
            self.data_folder += '/'

        self.G = nx.Graph()
        self.link_attr = pd.read_csv(self.data_folder + 'roads.csv')
        self.G.add_edges_from(self.link_attr[['snodeid', 'enodeid']].apply(tuple, axis=1))
        
        self.get_link_id = self.link_attr.set_index(['snodeid', 'enodeid'])['link_id'].to_dict()
        self.get_link_len = self.link_attr.set_index(['link_id'])['length'].to_dict()
        
        gps = pd.read_csv(self.data_folder + 'gps.csv')
        self.pos = gps.set_index('node_id')[['lon', 'lat']].apply(tuple, axis=1).to_dict()
        
        self.speed_data = dict()
        
    def shortest_path(self, start, dest, weight='length'):
        if weight not in self.link_attr:
            raise NotImplementedError()
        attr = self.link_attr[['snodeid', 'enodeid', 'length']].set_index(['snodeid', 'enodeid'])['length'].to_dict()
        nx.set_edge_attributes(self.G, attr, weight)
        
        return nx.shortest_path(self.G, start, dest, weight)
    
    def compute_time_last(self, timestamp, path):
        if isinstance(timestamp, str):
            timestamp = datetime.fromisoformat(timestamp)
        
        day = str(timestamp.date())
        if day not in self.speed_data:
            speed_df = pd.read_csv(self.data_folder + 'traffic/' + day, header=None)
            speed_df = speed_df.groupby([0, 1])[2].mean().unstack().T.sort_index()
            self.speed_data[day] = speed_df.to_dict('list')
        
        start_time_sec = (timestamp.hour * 60 + timestamp.minute) * 60 + timestamp.second
        if timestamp.hour > 23:
            raise NotImplementedError()
        
        t_idx = int(start_time_sec // (60 * 15)) - 1
            
        travel_time = 0
        for s, e in zip(path, path[1:]):
            if (s, e) in self.get_link_id:
                s_e = (s, e)
            else:
                s_e = (e, s)

            edge_id = self.get_link_id[s_e]
            length = self.get_link_len[edge_id]
            speed = self.speed_data[day][edge_id][t_idx]
            edge_time = length / speed * 60 * 60
            
            travel_time += edge_time
        return travel_time
    
    def compute_time_real(self, timestamp, path):
        if isinstance(timestamp, str):
            timestamp = datetime.fromisoformat(timestamp)

        day = str(timestamp.date())
        if day not in self.speed_data:
            speed_df = pd.read_csv(self.data_folder + 'traffic/' + day, header=None)
            speed_df = speed_df.groupby([0, 1])[2].mean().unstack().T.sort_index()
            self.speed_data[day] = speed_df.to_dict('list')
        
        start_time_sec = (timestamp.hour * 60 + timestamp.minute) * 60 + timestamp.second
        if timestamp.hour > 23:
            raise NotImplementedError()
        
        travel_time = 0
        for s, e in zip(path, path[1:]):
            if (s, e) in self.get_link_id:
                s_e = (s, e)
            else:
                s_e = (e, s)

            edge_id = self.get_link_id[s_e]
            edge_time = self._compute_edge_time(start_time_sec, edge_id, self.speed_data[day][edge_id])
            travel_time += edge_time
            start_time_sec += edge_time
        return travel_time
        
    def _compute_edge_time(self, start_time, edge, speed, length=None):
        '''
        Computes time to travel the edge. We consider two possible outcomes:
        1) edge is passed in one 15-minutes interval
        2) edge is passed in two consecutive 15-minutes interval
        And we hope that every edge could be passed that way)
        
        length :: kilometers
        speeds :: kmph - array of speed on that edge in start of 5 consecutive 15-minutes intervals
        start_time :: start time of edge in seconds. Relative to the start of the day
        '''
        
        # S = v0 * t + a * t^2 / 2
        # a * t^2 / 2 + v0 * t - S = 0
        # D = v0^2 + 4 * a/2 * S = v0^2 - 2 * a * S
        
        if length is None:
            length = self.get_link_len[edge]
        
        t_idx = int(start_time // (60 * 15))
        
        a = (speed[t_idx + 1] - speed[t_idx]) * 4
        v0 = speed[t_idx] + a * (start_time % (60 * 15)) / 60 / 60
        
        d = v0 * v0 + 4 * a * length
        assert d > 0
        
        root1 = (-v0 + sqrt(d)) / a / 2
        root2 = (-v0 - sqrt(d)) / a / 2
        
        err1 = length - v0 * root1 - a/2*root1*root1
        err2 = length - v0 * root2 - a/2*root2*root2
        
        if abs(err1) < abs(err2):
            time = root1
        else:
            time = root2

        assert min(map(abs, [err1, err2])) < 1e-2

        t_to_end = 60 * 15 - (start_time % (60 * 15))
        
        if time * 60 * 60 < t_to_end:
            return time * 60 * 60
        else:
            t_to_end_h = t_to_end / 60 / 60
            next_int = start_time + t_to_end
            new_l = length - (v0 * t_to_end_h + a / 2 * t_to_end_h * t_to_end_h)
            return t_to_end + self._compute_edge_time(next_int, edge, speed, new_l)

In [299]:
roads = Roads('data_clean')
sp = roads.shortest_path(1520482305, 1530638556)

print(roads.compute_time_real(datetime.fromisoformat('2017-04-01 10:14:59'), sp) / 60)
print(roads.compute_time_last(datetime.fromisoformat('2017-04-01 10:14:59'), sp) / 60)

14.939717584210625
13.638856273812797


10.27014

In [241]:
roads.compute_real_time(datetime.fromisoformat('2017-04-01 01:05:00'), sp)

0.17116980119282618

In [151]:
roads.link_attr.set_index(['snodeid', 'enodeid'])['link_id'].head().to_dict()

{(1520482305, 1554157827): 1004419350530,
 (1520491527, 1531173821): 1144134225924,
 (1520486999, 1530785480): 1144042225671,
 (1520486460, 1530638556): 1144134225930,
 (1520482294, 1520494440): 1462215565325}

In [172]:
roads.G[1520482305]

AtlasView({1554157827: {'length': 0.013}, 1520496758: {'length': 0.017}, 1520485301: {'length': 0.007}, 1520489945: {'length': 0.11}, 1520482952: {'length': 0.039}})

In [177]:
speed_df = pd.read_csv('data_clean' + '/traffic/2017-04-01', header=None)
speed_df = speed_df.groupby([0, 1])[2].mean().unstack().T.sort_index()
# speed_df.head().to_dict('list')

In [191]:
roads = Roads('data_clean')
sp = roads.shortest_path(1520482305, 1530638556)

# roads._compute_edge_time(100, 1004419350530, )

In [133]:
%%timeit
sp = roads.shortest_path(1520482305, 1530638556)

31.2 ms ± 665 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
from datetime import datetime

In [107]:
datetime.fromisoformat('2021-01-02 03:04:08').minute

4

In [116]:
tmp = pd.read_csv('data_clean/traffic/2017-04-05', header=None)
tmp.groupby([0, 1])[2].mean().unstack().min().min()

3.15799562222

In [58]:
ten = list(roads.G.nodes)[:10]
G_new = nx.subgraph(roads.G, ten)

In [89]:
roads.G.edges[(1520482305, 1554157827)]

{'length': 0.013}

In [78]:
list(roads.G.edges)[:10]

[(1520482305, 1554157827),
 (1520482305, 1520496758),
 (1520482305, 1520485301),
 (1520482305, 1520489945),
 (1520482305, 1520482952),
 (1554157827, 1520486069),
 (1520491527, 1531173821),
 (1520491527, 1520481541),
 (1520491527, 1530821415),
 (1520491527, 1520499272)]

In [93]:
h

[19.418999999999993,
 19.41999999999998,
 19.48199999999998,
 19.481999999999992,
 19.472999999999992,
 19.490999999999993,
 19.54499999999998,
 19.58599999999999,
 19.531999999999993,
 19.47699999999998,
 19.556999999999977]

In [92]:
h = []

from tqdm.auto import tqdm

for snode, end_dict in tqdm(nx.all_pairs_dijkstra_path_length(roads.G, weight='length')):
    for enode, l in end_dict.items():
        if len(h) > 10:
            heappushpop(h, l)
        else:
            heappush(h, l)

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [51]:
from heapq import heappush, heappushpop

h = []
heappush(h, 1)
heappush(h, 2)
heappush(h, 3)
heappush(h, 4)
heappush(h, 5)
heappush(h, 0)

In [52]:
h

[0, 2, 1, 4, 5, 3]

In [54]:
heappushpop(h, 10)

0

In [302]:
import spinup

ModuleNotFoundError: No module named 'spinup'

In [301]:
!python3 -m pip install spinningup

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
ERROR: Could not find a version that satisfies the requirement spinningup (from versions: none)
ERROR: No matching distribution found for spinningup
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
